In [1]:
!pip3 install numpy
!pip3 install absl-py
!pip3 install --upgrade tensorflow
!pip3 install "tensorflow>=2.0.0"
!pip3 install --upgrade tensorflow-hub
!pip3 install tensorflow_text
!pip3 install --ignore-installed Pillow==9.0.0


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip
  Using cached Pillow-9.0.0-cp39-cp39-macosx_10_10_x86_64.whl (3.0 MB)

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [43]:
# pip list

Package                      Version
---------------------------- -----------
absl-py                      1.1.0
alembic                      0.9.9
amqp                         2.3.2
anyio                        3.6.1
appnope                      0.1.0
asttokens                    2.0.5
astunparse                   1.6.3
backcall                     0.1.0
billiard                     3.5.0.4
blinker                      1.4
blis                         0.7.7
cachetools                   5.2.0
catalogue                    2.0.7
celery                       4.2.1
certifi                      2018.4.16
chardet                      3.0.4
charset-normalizer           2.1.0
click                        8.1.3
cycler                       0.11.0
cymem                        2.0.6
de-core-news-lg              3.3.0
de-core-news-md              3.3.0
debugpy                      1.6.0
decorator                    4.3.0
entrypoints                  0.4
executing                    0.8.3
fastapi  

In [44]:
# !pip3 freeze > requirements.txt

In [2]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re  # RegEx
# import seaborn as sns


import spacy
nlp = spacy.load('de_core_news_lg')

import xmltodict

In [3]:
import time

In [4]:
%%time
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3" 
#@param ["https://tfhub.dev/google/nnlm-de-dim50-with-normalization/2", 
# "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

2022-08-02 17:36:56.526470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


module https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3 loaded
CPU times: user 9.43 s, sys: 3.61 s, total: 13 s
Wall time: 1min 8s


In [6]:
%%time
skills = pd.read_csv('../data/skills_de.csv',usecols=['conceptUri','preferredLabel','description'])

CPU times: user 77.9 ms, sys: 19.9 ms, total: 97.9 ms
Wall time: 113 ms


In [7]:
skills.head(10)

,conceptUri,preferredLabel,description
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...
1,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...
2,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ..."
3,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...
5,http://data.europa.eu/esco/skill/000bb1e4-89f0...,toxikologische Studien durchführen,Durchführen von Tests zum Nachweis von Giften ...
6,http://data.europa.eu/esco/skill/000c94d2-2a2e...,Einheitlichkeit von Kokillen gewährleisten,Überwachen der Einheitlichkeit von Kokillen; V...
7,http://data.europa.eu/esco/skill/000f1d3d-220f...,Haskell,Techniken und Grundsätze der Softwareentwicklu...
8,http://data.europa.eu/esco/skill/001115fb-569f...,Initiative zeigen,Proaktiv sein und den ersten Schritt unternehm...
9,http://data.europa.eu/esco/skill/001d46db-035e...,Personal mit Blick auf die Verringerung von Le...,Einführung neuer Schulungs- und Personalentwic...


In [8]:
labels = skills['preferredLabel'].copy(deep=True)
labels

0                                  Musikpersonal verwalten
1                    Strafvollzugsverfahren beaufsichtigen
2                  nicht unterdrückende Praktiken anwenden
3        Einhaltung von Vorschriften von Eisenbahnfahrz...
4                             verfügbare Dienste ermitteln
                               ...                        
13886    berufliche Leistungsfähigkeit von Nutzern/Nutz...
13887             Beleuchtung in Transportgeräten einbauen
13888                     Verarbeitung natürlicher Sprache
13889                             Bauarbeiten koordinieren
13890         Absturzsicherungen und Bordbretter anbringen
Name: preferredLabel, Length: 13891, dtype: object

In [9]:
def labels_processing(text):
    processed = []
    for item in text:
        item = nlp(item)
        itemString = ''
        for word in item:
            word = word.lemma_.lower()
            if word != '--' and word != '' and word != ' ':
                if word == '\xa0': continue                    
                itemString += word + ' '
        processed.append(itemString[:-1])
    return processed

In [10]:
%%time
processedLabels = labels_processing(labels)

CPU times: user 1min 17s, sys: 785 ms, total: 1min 18s
Wall time: 1min 20s


In [42]:
%%time
test = open('../data/descriptions_cropped.xml','r', encoding='ISO-8859-15').read() # read data
test_dict = xmltodict.parse(xml_data) # parse xml
xmlDict = test_dict

CPU times: user 96.1 ms, sys: 34.7 ms, total: 131 ms
Wall time: 175 ms


In [12]:
%%time
xml_data = open('../data/FOKUS_AWT_CompetencyExtraction_WB_Brandenburg.xml','r', encoding='ISO-8859-15').read() # read data
xmlDict = xmltodict.parse(xml_data) # parse xml

CPU times: user 7.79 s, sys: 458 ms, total: 8.24 s
Wall time: 8.62 s


In [12]:
%%time
xmlcourses = xmlDict['DEFTISCAT']['COURSETRANSACTIONS']['INSERTCOURSES']['COURSE']

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 15.3 µs


In [13]:
%%time
coursesDict = []
for course in xmlcourses:
    coursesDict.append({"course_id": course['CS_ID'], 
                        "course_name": course['CS_NAME'],
                 "course_description": course['CS_DESC_LONG']})
courses_table = pd.DataFrame(coursesDict)
courses_table.replace([None],value='',inplace=True)
courses = courses_table['course_name'] + ' ' + courses_table['course_description']

CPU times: user 4.52 ms, sys: 2.16 ms, total: 6.68 ms
Wall time: 8.75 ms


In [14]:
courses_table

,course_id,course_name,course_description
0,A1E06AAB-F264-4308-F318-2C2E43E1CB54,Experte im Prozessmanagement,Die aktuell vorherrschende Situation auf dem A...
1,A1DFC244-E07B-BE74-4422-1995EE3B5F8A,Experte in Cloud Computing,"""Cloud Computing"" ist die bedarfsabhängige Ber..."
2,A1E08F41-0B47-7D51-C6F5-47D497F050F0,Experte in Investition und Finanzierung,Die aktuell vorherrschende Situation auf dem A...
3,A1DFF789-1B71-1CD5-F972-9AA4F3B32C8B,Experte in Unternehmensaufbau und Organisation,Der Kurs zielt auf die Verbesserung der Führun...
4,A1E05E65-48FD-DE85-520D-283A9117BCD9,Experte in Unternehmensführung,Die aktuell vorherrschende Situation auf dem A...
5,A1E0969B-12DB-B016-964F-919D1DEB0112,IT-Projektmanagement,Die aktuell vorherrschende Situation auf dem A...
6,A1DFDEA2-E36E-E91A-D9B8-3F0956BECDE6,PHP Programmierer und Datenbankentwickler,PHP (Hypertext Preprocessor) ist eine quelloff...
7,A1DFEB90-BC08-8DA2-A8A2-30C4C2E6E989,"Programmierung PHP Frameworks: Laravel, Symfon...",PHP (Hypertext Preprocessor) ist eine quelloff...
8,A1DF7719-5965-53A1-C4CF-22D1AEE3D88B,Shopsystem Administrator,Ziel der Maßnahme ist es den Teilnehmern eine ...
9,A1DF94D9-0547-7ED8-AB3C-6233DFB724AE,Social-Media Manager,Ziel der Maßnahme ist es den Teilnehmern eine ...


In [15]:
courses

0     Experte im Prozessmanagement Die aktuell vorhe...
1     Experte in Cloud Computing "Cloud Computing" i...
2     Experte in Investition und Finanzierung Die ak...
3     Experte in Unternehmensaufbau und Organisation...
4     Experte in Unternehmensführung Die aktuell vor...
5     IT-Projektmanagement Die aktuell vorherrschend...
6     PHP Programmierer und Datenbankentwickler PHP ...
7     Programmierung PHP Frameworks: Laravel, Symfon...
8     Shopsystem Administrator Ziel der Maßnahme ist...
9     Social-Media Manager Ziel der Maßnahme ist es ...
10    Starte jetzt: Heranführung an die selbstständi...
11    Vermarktungsstrategien online und offline Die ...
12    Webentwicklung 2.0 - HTML5, CSS3, WordPress Zi...
13    Zukunftswert Mensch: tiefgreifend-wirksame ind...
14    Weiterbildung Wildnispädagogik Wir freuen uns ...
15    Qualifizierung zur Anerkennung für Erzieher*in...
16    Digitales spielbasiertes Lernen - Einsatzmögli...
17    Digitales spielbasiertes Lernen - Einsatzm

In [16]:
length = len(courses)
length

19

In [17]:
%%time
for i in range(length):
    courses[i] = courses[i].replace('&#8211;','-')
    courses[i] = courses[i].replace('&#8222;','„')
    courses[i] = courses[i].replace('&#8220;','“')
    courses[i] = courses[i].replace('&#8230;','...')

CPU times: user 1.12 ms, sys: 2 µs, total: 1.13 ms
Wall time: 1.13 ms


In [18]:
nlp.add_pipe('merge_noun_chunks')

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [19]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner', 'merge_noun_chunks']


In [20]:
%%time
def To_sentences(courses):
    courses_sent = []
    courses_sent_lemma = []
    for course in (courses):
        with nlp.select_pipes(disable=['merge_noun_chunks','attribute_ruler', 'ner']):
            doc = nlp(str(course))
            sentences = []
            for sent in doc.sents:
                course_processed = ''
                sents = []
                for word in sent:
                    word = word.text
                    word = word.replace('\n', ' ')
                    word = word.strip().strip('"@#$%^&*§')
                    if word == '-':
                        if course_processed != '':
                            sents.append(course_processed[:-1])
                            course_processed = ''
                        continue
                    if word.startswith('- ') and len(word) > 1:
                        word = word.replace('- ', '')
                        if course_processed != '':
                            sents.append(course_processed[:-1])
                            course_processed = ''
                    if (not re.match('\s+', word,re.I)) and word != '--' and word != '':
                        if "/-" in word: 
                            word = word.split("/-")[0]
                        if re.match(r'\,|\.|\?|\!|\)', word):
                            course_processed = course_processed[:-1] + word + ' '
                        elif re.match(r'\(',word):
                            course_processed += word
                        else:
                            course_processed += word + ' '
                sentences += sents
                if course_processed != '':
                    sentences.append(course_processed[:-1])
            courses_sent_lemma.append(sentences)
    return courses_sent_lemma

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


In [21]:
course_in_sentences_test = To_sentences(courses.iloc[:1])
print(courses.iloc[0])
course_in_sentences_test

Experte im Prozessmanagement Die aktuell vorherrschende Situation auf dem Arbeitsmarkt gestaltet sich als wandelbar und schnelllebig. Globalisierung und Technisierung tragen einen großen Anteil dazu bei. Verschiedene Kulturen treffen im Arbeitsleben aufeinander. Verschiedene Abteilungen und Aufgabenbereiche treffen aufeinander. Es wird immer komplexer.

Ziel der Maßnahme Weiterbildung Prozessmanagement ist es deshalb, den Teilnehmenden eine solide Basis auf dem Gebiet des Prozessmanagements zu vermitteln. Welche Fach- und Wissensbereiche dabei unabdingbar sind, haben wir in der Kurzbeschreibung dargestellt.

Unser Verständnis davon, zukünftige Spezialisten im Bereich des Prozessmanagements auszubilden, hat weniger damit zu tun, theoretisches Wissen zu vermitteln, sondern unsere Teilnehmer von Weiterbildung Prozessmanagement in die Lage zu versetzen, als professionelle Prozessmanager in der Praxis auftreten zu können, indem wir ihnen die richtigen Werkzeuge dafür an die Hand geben. Das 

[['Experte im Prozessmanagement',
  'Die aktuell vorherrschende Situation auf dem Arbeitsmarkt gestaltet sich als wandelbar und schnelllebig.',
  'Globalisierung und Technisierung tragen einen großen Anteil dazu bei.',
  'Verschiedene Kulturen treffen im Arbeitsleben aufeinander.',
  'Verschiedene Abteilungen und Aufgabenbereiche treffen aufeinander.',
  'Es wird immer komplexer.',
  'Ziel der Maßnahme',
  'Weiterbildung Prozessmanagement ist es deshalb, den Teilnehmenden eine solide Basis auf dem Gebiet des Prozessmanagements zu vermitteln.',
  'Welche Fach- und Wissensbereiche dabei unabdingbar sind, haben wir in der Kurzbeschreibung dargestellt.',
  'Unser Verständnis davon, zukünftige Spezialisten im Bereich des Prozessmanagements auszubilden, hat weniger damit zu tun, theoretisches Wissen zu vermitteln, sondern unsere Teilnehmer von Weiterbildung Prozessmanagement in die Lage zu versetzen, als professionelle Prozessmanager in der Praxis auftreten zu können, indem wir ihnen die ric

In [22]:
%%time
def Segment_processing(part_of_the_courses):
    
    # to sentences
    course_in_sentences = To_sentences(part_of_the_courses)
    
    # to chunks
    courses_chunks = []
    for course in course_in_sentences:
        course_chunks = []
        for sent in course:
            sent_chunks = []
            doc = nlp(sent)
            for tok in doc:
                with nlp.select_pipes(disable=['merge_noun_chunks','attribute_ruler', 'ner']):
#                     with nlp.select_pipes(disable=['attribute_ruler', 'ner']):
                    if not re.match('\(|\)|\,|\.|\!|\?|\/',tok.text) and not tok.is_stop: 
                        tok = nlp(tok.text)
                        words = ''
                        for word in tok:
                            if not word.is_stop and word.lemma_ != '--':
                                words = words + word.lemma_ + ' '  
                        sent_chunks.append(words[:-1])
            course_chunks += sent_chunks
        courses_chunks.append(course_chunks)
    return courses_chunks


CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 13.1 µs


In [23]:
%%time
courses_chunks_test = Segment_processing(courses.iloc[:1])
print(course_in_sentences_test)
print(courses_chunks_test)

[['Experte im Prozessmanagement', 'Die aktuell vorherrschende Situation auf dem Arbeitsmarkt gestaltet sich als wandelbar und schnelllebig.', 'Globalisierung und Technisierung tragen einen großen Anteil dazu bei.', 'Verschiedene Kulturen treffen im Arbeitsleben aufeinander.', 'Verschiedene Abteilungen und Aufgabenbereiche treffen aufeinander.', 'Es wird immer komplexer.', 'Ziel der Maßnahme', 'Weiterbildung Prozessmanagement ist es deshalb, den Teilnehmenden eine solide Basis auf dem Gebiet des Prozessmanagements zu vermitteln.', 'Welche Fach- und Wissensbereiche dabei unabdingbar sind, haben wir in der Kurzbeschreibung dargestellt.', 'Unser Verständnis davon, zukünftige Spezialisten im Bereich des Prozessmanagements auszubilden, hat weniger damit zu tun, theoretisches Wissen zu vermitteln, sondern unsere Teilnehmer von Weiterbildung Prozessmanagement in die Lage zu versetzen, als professionelle Prozessmanager in der Praxis auftreten zu können, indem wir ihnen die richtigen Werkzeuge d

## Embedding-phase

In [24]:
def Course_Embedding(courses_chunks):
    courses_chunks_embed = []
    for course in courses_chunks:
        courses_chunks_embed.append(embed(course))
    return courses_chunks_embed

In [30]:
%%time
labels_embed = embed(processedLabels)

CPU times: user 1min 58s, sys: 36.1 s, total: 2min 34s
Wall time: 32.7 s


## Similarity

In [31]:
%%time
def Similarity(labels, courses):
    result = []
    for course in courses:
        result.append(np.dot(labels, np.transpose(course)))
    return result

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 6.91 µs


In [38]:
%%time
def Get_relations(relations,skills,courses_table,course_idx,benchmark,courses_chunks):

    idx = 0
    counter = 0

    df_relations = pd.DataFrame(columns = ['conceptUri'])
    for relationship in relations:
        print('course: ', course_idx)
        competency_idx = np.where(relationship>benchmark) 

        print('numbers of extracted competency: ', len(competency_idx[0]))
        for j in range(len(competency_idx[0])):
            print(processedLabels[competency_idx[0][j]],'--', courses_chunks[idx][competency_idx[1][j]])
            
        competency = set(skills.loc[list(competency_idx[0]),'conceptUri'])
        competency_uri = pd.DataFrame(competency,columns=['conceptUri'], index=[courses_table.iloc[course_idx]['course_id']]*len(competency)) 

        print(' ------')
#         print(courses[course_idx])
        print('\n')

        df_relations = pd.concat([df_relations,competency_uri])
    
        course_idx += 1 
        
#         test:
        idx += 1
        counter += len(competency)
    print('number of extracted competency', counter)
    print('\n', df_relations)

    return df_relations

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 14.1 µs


In [33]:
def Competence_extraction(courses,course_start_idx,labels,benchmark):
    courses_chunks = Segment_processing(courses)
    courses_embed = Course_Embedding(courses_chunks)
    calculated_results = Similarity(labels, courses_embed)
    return Get_relations(calculated_results,skills,courses_table,course_start_idx,benchmark,courses_chunks)

In [29]:
%%time
def Save_results(courses,benchmark):
    for i in range(27):
        start = i * 600
        end = (i+1) * 600
        courses_div = courses.iloc[start:end]
        relations = Competence_extraction(courses_div,start,labels_embed,benchmark)
        relations.to_csv(path_or_buf="./relations_part_"+str(i+1)+".csv" ,columns=['conceptUri'], index_label='course_id')
    Competence_extraction(courses.iloc[27*600:],27*600,labels_embed,benchmark).to_csv(path_or_buf="./relations_part_28.csv" ,columns=['conceptUri'], index_label='course_id')


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [39]:
%%time
Competence_extraction(courses,0,labels_embed,benchmark=0.83)

course:  0
numbers of extracted competency:  15
probe organisieren -- organisieren
ernte organisieren -- organisieren
stellenanalysen durchführen -- analysieren
organisieren arbeiten -- organisieren
darlehen analysieren -- analysieren
auftritt organisieren -- organisieren
arbeitsmarktangebot -- Arbeitsmarkt
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
arbeitsmarkt -- Arbeitsmarkt
 ------


course:  1
numbers of extracted competency:  3
datenbank führen -- Datenbanke
datenbank -- Datenbanke
internet der ding -- Internet
 ------


course:  2
numbers of extracted competency:  4
finanzmarktgeschaft durchführen -- Finanzmarkt
arbeitsmarktangebot -- Arbeitsmarkt
finanzmarkt -- Finanzmarkt
arb

,conceptUri
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/38a5c459-c591...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/d9a3cb06-d6e0...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/d5a59ca8-2e91...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/0fbaceff-df05...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/06bf62ec-3316...
...,...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/485cdafb-3c7f...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/57437c77-18a4...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/47b49f99-74a3...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/c9f4efcd-e16a...


In [41]:
%%time
Competence_extraction(courses,0,labels_embed,benchmark=0.825)

course:  0
numbers of extracted competency:  17
probe organisieren -- organisieren
ernte organisieren -- organisieren
stellenanalysen durchführen -- analysieren
organisieren arbeiten -- organisieren
darlehen analysieren -- analysieren
auftritt organisieren -- organisieren
arbeitsmarktangebot -- Arbeitsmarkt
unternehmensinkubation -- Unternehmen
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
prozessorientiert management -- Prozessmanagement
arbeitsmarkt -- Arbeitsmarkt
aufführungsstätt organisieren -- organisieren
 ------


course:  1
numbers of extracted competency:  3
datenbank führen -- Datenbanke
datenbank -- Datenbanke
internet der ding -- Internet
 ------


course:  2
numbers of extracted competency:  5
finanzmarktgeschaft durchführ

,conceptUri
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/61ddacb8-c89f...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/38a5c459-c591...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/e5e48620-8476...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/d9a3cb06-d6e0...
A1E06AAB-F264-4308-F318-2C2E43E1CB54,http://data.europa.eu/esco/skill/d5a59ca8-2e91...
...,...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/57437c77-18a4...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/47b49f99-74a3...
A0CA8AEC-98C1-1D6F-1831-FFBC9DE00861,http://data.europa.eu/esco/skill/c9f4efcd-e16a...
A1E00470-9251-11EC-AB1A-357BCD0E40C4,http://data.europa.eu/esco/skill/7c3cde53-9ff7...
